In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mplcursors
%matplotlib notebook
%matplotlib widget
import pandas as pd
from matplotlib.lines import Line2D
from scipy.stats import linregress 
from scipy.fft import fft, rfft,fftshift
from scipy.fft import fftfreq, rfftfreq

from scipy.fft import fft, rfft,fftshift
from scipy.fft import fftfreq, rfftfreq
import sys
import utilities
import importlib
importlib.reload(utilities)
import json

In [ ]:
pd.option_context('display.precision', 10)
pd.set_option('display.float_format', '{:.10f}'.format)

In [ ]:
graph_number=181

In [ ]:
first_line = pd.read_csv("classified_lcs\grs1915_lc"+str(graph_number)+".txt", nrows=1, header=None).iloc[0, 0]

# Extract the part after "# class : "
class_name = first_line.split("# class : ")[-1] if "# class : " in first_line else None

print(class_name)


In [ ]:
df=pd.read_csv("classified_lcs\grs1915_lc"+str(272)+".txt",sep="	",skiprows=[0, 1], header=None)

In [ ]:
n=len(df)

In [ ]:
df.columns = ['time', 'total','low','mid','high'] #time in seconds

In [ ]:
low=df["low"]

In [ ]:
plt.figure()
plt.plot(low)
plt.show()

In [ ]:
df

In [ ]:
df["total"] - (df["low"] + df["mid"])

In [ ]:
#time step is 0.125 sec

In [ ]:
low=np.array(list(df["low"]))
mid=np.array(list(df["mid"]))
high=np.array(list(df["high"]))

divide by 8 !!! to get the time in seconds

In [ ]:
plt.figure()  # Start a new figure

plt.plot(np.array(range(len(low))),low)
plt.xlim(294,294+597)
plt.show()

In [ ]:
plt.figure()  # Start a new figure
plt.plot(np.array(range(len(low))),low)
plt.xlim(294,890)
plt.show()

In [ ]:
# Apply the FFT on the signal
plt.figure()  # Start a new figure

fourier = fftshift(fft(low))

freqs = np.fft.fftshift(np.fft.fftfreq(len(low)))  # Normalized frequencies

# Plot the magnitude spectrum
plt.clf()
plt.plot(freqs, np.abs(fourier))
plt.xlim(-0.01,0.01)
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.title("Centered FFT Spectrum")
plt.show()
"""
# Plot the result (the spectrum |Xk|)
plt.plot(np.abs(fourier))
#plt.xlim(0,200)
plt.show()"""


getting the graph of distance comparison with the shapelet

saving the shaplet

In [ ]:
shaplet=low[294:891]

In [ ]:
#import pickle
#with open("shaplet.pkl", "wb") as f:
#    pickle.dump(shaplet, f)

In [ ]:
l=len(shaplet)

In [ ]:
l

In [ ]:
plt.figure()
plt.plot(shaplet)
plt.show()

In [ ]:
difference=[] #diffrence between shaplet & signal
mean_=np.mean(shaplet)
shapletlength=len(shaplet)
for i in range(0,len(low)-l):
    diff=np.sum(np.power(low[i:i+l]-shaplet,2) / (2*mean_*8*len(shaplet)) )
    difference.append(diff)
difference=np.array(difference)

In [ ]:
plt.figure()  # Start a new figure
plt.plot(range(len(difference)),difference)
plt.show()

In [ ]:
chi2=[] #diffrence between shaplet & signal
mean_=np.mean(shaplet)
for i in range(0,len(low)-l):
    diff=np.sum( np.power(low[i:i+l]-shaplet,2) / (2*mean_*8) )
    chi2.append(diff)
#chi2=chi2/(2*np.mean(shaplet))
chi2=np.array(chi2)

plt.figure()  
plt.plot(range(len(chi2)),chi2)
plt.show()
plt.figure()  # Start a new figure
plt.plot(range(len(chi2)),chi2/len(shaplet))
plt.axhline(y=1, color='r', linestyle='--', linewidth=2, label=f'y={1}')
plt.axhline(y=4, color='b', linestyle='--', linewidth=2, label=f'y={4}')
plt.axhline(y=1+np.sqrt(2/500), color='#FFA500', linestyle='--', linewidth=2, label=f'y={1+np.sqrt(2/500)}')
plt.show()

In [ ]:
import scipy.signal as sig
print(sig.argrelmin(difference,order=300))

In [ ]:
was_below=False
threshold=4
minima=1000
index=-1
minimaList=[] # list of (min,index)
print(chi2)
for i in range(len(chi2)):
    if(chi2[i]<threshold):
        if(chi2[i]<minima):
            minima=chi2[i]
            index=i
        was_below=True
    else:
        if(was_below):
            was_below=False
            minimaList.append((minima,index))
            minima=1000
            index=-1
        
print(minimaList)     


In [ ]:
len(minimaList)

In [ ]:
newList=sig.argrelmin(difference,order=300)[0]

In [ ]:
plt.figure()
plt.plot(range(len(low)),low,color='b') 
for el in newList:
    index=el
    plt.plot(range(index,index+len(shaplet)),shaplet,color='r',alpha=0.5)

plt.show()

In [ ]:
plt.figure()
plt.plot(shaplet)
plt.show()

Shaplet test for all data

In [ ]:
dist_table=[]
min_hist=[] #list of triplets of (index,min_diff
differences=[]#table of the table of difference; NOT COMPUTED YET
short_=[]
for i in range(571):
    #get class name:
    first_line = pd.read_csv("classified_lcs\grs1915_lc"+str(i)+".txt", nrows=1, header=None).iloc[0, 0]

    # Extract class name
    prefix = "# class : "
    class_name = first_line[len(prefix):].strip()


    #read table:
    df=pd.read_csv("classified_lcs\grs1915_lc"+str(i)+".txt",sep="	",skiprows=[0, 1], header=None)
    df.columns = ["time", 'total','low','mid','high'] #time in seconds
    low=np.array(list(df["low"]))

    dist_table.append(utilities.chi2dist(shaplet,low))
    
    try:
        min_hist.append(min(dist_table[-1]))
    except Exception as e:
        print("An error occurred:", e)
        


In [ ]:
min_hist=np.array(min_hist)

In [ ]:
plt.figure()
plt.hist(min_hist[min_hist<20],bins="fd")
plt.show()

In [ ]:
plt.figure()
plt.plot(dist_table[181])
plt.show()

In [ ]:
min_hist[np.argsort(min_hist)[:11]]

In [ ]:
np.argsort(min_hist)[:11]

In [ ]:
min_=[]
for d in dist_table:
    if(len(d)!=0):
        min_.append(min(d))
    else:
        min_.append(np.nan)


In [ ]:
L=np.argsort(min_)

In [ ]:
L

In [ ]:
mins=[]
for i in range(len(L)):
    j=len(L)-1-i
    if(not np.isnan(min_[L[j]])):
        mins=L[0:j+1]
        break

In [ ]:
mins

In [ ]:
with open('key2class.json', 'r') as f:
    data = json.load(f)

class_ = {int(k): v for k, v in data.items()}

In [ ]:
def plot_curve(i): #takes the index following mins
    plt.close("all")
    n=len(mins)
    lc_number=mins[i]
    lc_class=class_[lc_number]
    lc_min=min_[mins[i]]
    df=pd.read_csv("classified_lcs\grs1915_lc"+str(lc_number)+".txt",sep="	",skiprows=[0, 1], header=None)
    df.columns = ["time", 'total','low','mid','high'] #time in seconds
    low=np.array(list(df["low"]))
    plt.figure()
    
    legend="Class: "+str(lc_class)+"\nIndex: "+str(i)+"\nLC Number:"+str(lc_number)+"\ndist minima: "+str(lc_min)
    utilities.draw(shaplet,dist_table[lc_number],low,txt=legend)
    #plt.clf()
    print("class: ",lc_class)
    print("index: ",i)
    print("lc number: ",lc_number)

In [ ]:
%matplotlib inline
plt.clf()
plt.figure()
for i in range(0,50):
    plot_curve(i)

In [ ]:
%matplotlib inline
  
for j in range(87,87+1):
    plot_curve(i)

In [ ]:
len(utilities.get_low(280))

In [ ]:
plt.close('all')

In [ ]:
n

In [ ]:
sig.argrelmin(dist_table[lc_number],order=300)[0]

In [ ]:
n/16

### 

In [ ]:
%matplotlib widget
plot_curve(6)

In [ ]:
# Data dictionary (same structure as np.savez)
data = {
    "min_distances_table": min_,
    "argsort_min": L,
    "argsort_min_without_nan": mins,
    "dist_table": dist_table
}

# Save to a .pkl file
with open("computed_data.pkl", "wb") as f:
    pickle.dump(data, f)

In [ ]:
description = """This file contains:
- dist_table: is a table that contains distance array between the shaplet and each light_curve for each element
- min_distances_table: contains the minimum of each dist_table,
            \nif an element of dist table is empty,meaning the original curve is shorter than the shaplet
            \nthen the default value is np.nan
- argsort_min: the indeces sorted by the value of the min,it takes also indeces for which the minima is np.nan
- argsort_min_without_nan: same as argsort_min but without the nan elements
"""

# Save metadata
with open("arrays_info.txt", "w") as f:
    f.write(description)


In [ ]:
"""
with open("computed_data.pkl", "rb") as f:
    loaded_data = pickle.load(f)

# Access variables
min_loaded = loaded_data["min_distances_table"]
Lloaded = loaded_data["argsort_min"]
minsloadd = loaded_data["argsort_min_without_nan"]
dist_tableloaded = loaded_data["dist_table"]
"""


In [ ]:
?utilities.dist

<b style="color:dark;">Minimul resemblance visualization for each light curve

In [ ]:
importlib.reload(utilities)
def plot_at_high_resemblance(i):
    plt.close("all")
    n=len(mins)
    lc_number=mins[i]
    lc_class=class_[lc_number]
    lc_min=min_[mins[i]]
    print(lc_min,lc_class,lc_number)
    df=pd.read_csv("classified_lcs\grs1915_lc"+str(lc_number)+".txt",sep="	",skiprows=[0, 1], header=None)
    df.columns = ["time", 'total','low','mid','high'] #time in seconds
    low=np.array(list(df["low"]))

    minima_coordinates=np.where(dist_table[lc_number] == lc_min)[0][0]
    xlim=(minima_coordinates,minima_coordinates+len(shaplet))
#    print(xlim)
    plt.figure()
    title="Class: "+str(lc_class)+" LC Number:"+str(lc_number)+" dist minima: "+"{:.2f}".format(lc_min)
    utilities.draw(shaplet,dist_table[lc_number],low,txt=title,xlim=xlim)
#    utilities.draw(shaplet,dist_table[lc_number],low,txt=title)
    #plt.clf()
    print("class: ",lc_class)
    print("index: ",i)
    print("lc number: ",lc_number)

In [ ]:
%matplotlib inline
plot_at_high_resemblance(0) #order of the least distant to the farest distant,

In [ ]:
%matplotlib inline
for i in range(0,50):
    plot_at_high_resemblance(i)

In [ ]:
plt.hist(np.array(min_)[np.array(min_)<10],bins="fd")

<b style="color:red;">Get all minimas and save in a list of elements with element beeing: (i,coordinate,chi2)

In [ ]:
minimas_table=[]

In [ ]:
for i in range(len(dist_table)):
    minimas_coordinates=utilities.get_minimas_coordinates(shaplet,dist_table[i],300)
    for index in minimas_coordinates:
        minimas_table.append( [i,index, dist_table[i][index] ] )

In [ ]:
minimas_table

In [ ]:
minimas_table=np.array(minimas_table)

In [ ]:
minimas_table_sorted = minimas_table[minimas_table[:, 2].argsort()] #SAVE THIS LIST !!!!!!

In [ ]:
minimas_table= np.array(minimas_table_sorted)

In [ ]:
minimas_table_sorted[0]

In [ ]:
def plot_(curve,xlim):
    plt.plot(curve)
    plt.xlim(xlim,xlim+len(shaplet))
    plt.show()

In [ ]:
plt.figure()
plt.hist(minimas_table[:, 2][minimas_table[:, 2]<25],bins=np.arange(0,10));
plt.show()

In [ ]:
mylist=minimas_table[ abs(minimas_table[:, 2] - 2) < 0.5 ]

In [ ]:
len(mylist)

In [ ]:
%matplotlib inline
plt.close("all")

for el in mylist:
    index=int(el[0])
    co=int(el[1])
    chi2=el[2]
    df=pd.read_csv("classified_lcs\grs1915_lc"+str(int(index))+".txt",sep="	",skiprows=[0, 1], header=None)
    df.columns = ["time", 'total','low','mid','high'] #time in seconds
    low=np.array(list(df["low"]))

    utilities.draw(shaplet,dist_table[index],low,txt="chi2: " + str(chi2) + ", class: "+str(class_[index]),xlim=(co,co+len(shaplet)))

In [ ]:
local_minimas_table_sorted = minimas_table_sorted

first_column_local_minimas_table_sorted = [item[0] for item in local_minimas_table_sorted]
second_column_local_minimas_table_sorted = [item[1] for item in local_minimas_table_sorted]
third_column_local_minimas_table_sorted = [item[2] for item in local_minimas_table_sorted]

# Step 1: Load the existing data (if any) from the Pickle file "computed_data"
with open("computed_data.pkl", "rb") as f:
    data = pickle.load(f)  # Load the existing data (it can be a dictionary)


# Step 2: Separate the columns from the new data
first_column_local_minimas_table_sorted = [item[0] for item in local_minimas_table_sorted]
second_column_local_minimas_table_sorted = [item[1] for item in local_minimas_table_sorted]
third_column_local_minimas_table_sorted = [item[2] for item in local_minimas_table_sorted]

# Step 3: Update the data (add the new columns)
data['first_column'] = first_column_local_minimas_table_sorted
data['second_column'] = second_column_local_minimas_table_sorted
data['third_column'] = third_column_local_minimas_table_sorted

# Step 4: Save the updated data back to the same Pickle file "computed_data"
with open("computed_data.pkl", "wb") as f:
    pickle.dump(data, f)

# Check the final data
print(data)


In [ ]:
data.keys()

In [ ]:
data["first_column"]

In [ ]:
#testing
with open("computed_data.pkl", "rb") as f:
    loaded = pickle.load(f)  # Load the existing data (it can be a dictionary)


In [ ]:
loaded.keys()